In [27]:
import pandas as pd
from alpha_vantage.timeseries import TimeSeries
import time
import matplotlib
import matplotlib.pyplot as plt
from datetime import datetime

In [28]:
btc_min_data = pd.read_csv('/Users/ashutoshbhawsar/Documents/Fall 2022 Courses/DIC/DIC project/bitstampUSD_1-min_data_2012-01-01_to_2021-03-31.csv') 

print(btc_min_data)

          Timestamp      Open      High       Low     Close  Volume_(BTC)  \
0        1325317920      4.39      4.39      4.39      4.39      0.455581   
1        1325317980       NaN       NaN       NaN       NaN           NaN   
2        1325318040       NaN       NaN       NaN       NaN           NaN   
3        1325318100       NaN       NaN       NaN       NaN           NaN   
4        1325318160       NaN       NaN       NaN       NaN           NaN   
...             ...       ...       ...       ...       ...           ...   
4857372  1617148560  58714.31  58714.31  58686.00  58686.00      1.384487   
4857373  1617148620  58683.97  58693.43  58683.97  58685.81      7.294848   
4857374  1617148680  58693.43  58723.84  58693.43  58723.84      1.705682   
4857375  1617148740  58742.18  58770.38  58742.18  58760.59      0.720415   
4857376  1617148800  58767.75  58778.18  58755.97  58778.18      2.712831   

         Volume_(Currency)  Weighted_Price  
0                 2.000000    

In [29]:

print(btc_min_data.columns)

Index(['Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume_(BTC)',
       'Volume_(Currency)', 'Weighted_Price'],
      dtype='object')


In [30]:
# convert unix time to timestamp
btc_min_data['Timestamp']=pd.to_datetime(btc_min_data['Timestamp'],unit='s')

In [31]:
btc_min_data

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
0,2011-12-31 07:52:00,4.39,4.39,4.39,4.39,0.455581,2.000000,4.390000
1,2011-12-31 07:53:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2011-12-31 07:54:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2011-12-31 07:55:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2011-12-31 07:56:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
4857372,2021-03-30 23:56:00,58714.31,58714.31,58686.00,58686.00,1.384487,81259.372187,58692.753339
4857373,2021-03-30 23:57:00,58683.97,58693.43,58683.97,58685.81,7.294848,428158.146640,58693.226508
4857374,2021-03-30 23:58:00,58693.43,58723.84,58693.43,58723.84,1.705682,100117.070370,58696.198496
4857375,2021-03-30 23:59:00,58742.18,58770.38,58742.18,58760.59,0.720415,42332.958633,58761.866202


In [32]:
# Drop all Not-a-number(NaN) or NA values
btc_min_data.dropna()

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
0,2011-12-31 07:52:00,4.39,4.39,4.39,4.39,0.455581,2.000000,4.390000
478,2011-12-31 15:50:00,4.39,4.39,4.39,4.39,48.000000,210.720000,4.390000
547,2011-12-31 16:59:00,4.50,4.57,4.50,4.57,37.862297,171.380338,4.526411
548,2011-12-31 17:00:00,4.58,4.58,4.58,4.58,9.000000,41.220000,4.580000
1224,2012-01-01 04:16:00,4.58,4.58,4.58,4.58,1.502000,6.879160,4.580000
...,...,...,...,...,...,...,...,...
4857372,2021-03-30 23:56:00,58714.31,58714.31,58686.00,58686.00,1.384487,81259.372187,58692.753339
4857373,2021-03-30 23:57:00,58683.97,58693.43,58683.97,58685.81,7.294848,428158.146640,58693.226508
4857374,2021-03-30 23:58:00,58693.43,58723.84,58693.43,58723.84,1.705682,100117.070370,58696.198496
4857375,2021-03-30 23:59:00,58742.18,58770.38,58742.18,58760.59,0.720415,42332.958633,58761.866202


In [34]:
btc_min_data.groupby('Timestamp')['Low'].min()

Timestamp
2011-12-31 07:52:00        4.39
2011-12-31 07:53:00         NaN
2011-12-31 07:54:00         NaN
2011-12-31 07:55:00         NaN
2011-12-31 07:56:00         NaN
                         ...   
2021-03-30 23:56:00    58686.00
2021-03-30 23:57:00    58683.97
2021-03-30 23:58:00    58693.43
2021-03-30 23:59:00    58742.18
2021-03-31 00:00:00    58755.97
Name: Low, Length: 4857377, dtype: float64

In [37]:
# Temp convert timestamp to date
btc_min_data['Timestamp']=btc_min_data['Timestamp'].date()

AttributeError: 'Series' object has no attribute 'date'

In [4]:
def get_combined_daily_digi_currencies(ts_list):
    """
    Get digital currencies' daily data and combine them into one dataframe
    """
    from alpha_vantage.cryptocurrencies import CryptoCurrencies
    CC = CryptoCurrencies(key='19P74MEDAA0IADOG', output_format= 'pandas')
    #data_tmp = pd.DataFrame(0 ,  columns = assets)  
    d = []

    for s in ts_list:
        data, meta_data = CC.get_digital_currency_daily(symbol=s, market='USD')
        data.rename(columns = {"4b. close (USD)": s}, inplace  = True)
        d.append(data[s])
    
    df = pd.concat(d, axis=1)
    return df

In [5]:
digi_currencies = ['BTC', # Bitcoin
                   'LTC', # Litecoin
                   'DASH', # Dash
                   'XEM', # NEM
                   'ETH', # Ethereum    
                  ]

instr = digi_currencies

price = get_combined_daily_digi_currencies(instr)
price

,BTC,LTC,DASH,XEM,ETH
date,,,,,
2020-01-14,8810.01,58.39,89.28,NaN,165.64
2020-01-15,8821.41,58.16,133.60,NaN,166.40
2020-01-16,8720.01,57.56,127.45,NaN,164.21
2020-01-17,8913.28,61.02,113.04,NaN,169.85
2020-01-18,8915.96,59.32,101.15,NaN,174.14
...,...,...,...,...,...
2022-10-05,20158.26,54.63,42.15,0.0416,1352.20
2022-10-06,19960.67,53.53,42.06,0.0409,1352.06
2022-10-07,19530.09,52.83,42.28,0.0407,1331.10
